In [2]:
import multiprocessing as mp
import pandas as pd
import numpy as np
import time
import datetime
from tqdm import tqdm
from tqdm import trange 
import multiprocessing
from func import *
import os

In [3]:
data= pd.read_csv(r'group_call_HMS.csv')

In [ ]:
print(data.head())

   call_start_day call_start_time  call_end_day call_end_time  \
0               1    09:14:58.558             1  09:41:30.200   
1              17    11:05:05.176            17  13:07:42.515   
2              20    02:18:43.172            20  02:28:58.177   
3              22    09:22:01.936            22  09:47:40.192   
4              26    06:29:21.182            26  06:50:55.004   

                                              hashed  
0  967393e81d99ce8e577ee130b7ce8e4fd45e3e9cecb560...  
1  02181a0c962f34f019bc9d5b582fb0ec79b1441f96aa4d...  
2  86022904c5cf72a54978479c94041f4256d6c3c2a1f71c...  
3  aafb40d212fe18ff4eafb82fdcf3b53f2161cb3ce59de4...  
4  c87c2fad141edf323f3787335b54be22945a02fe052448...  


In [4]:
data['hashed'][0]

'967393e81d99ce8e577ee130b7ce8e4fd45e3e9cecb560de427ede6ea49e024f,a0b6ecbec654b18fe36ebe6230e25a653fb12125733583d012741572134447f4,3193ab18168bcadbcb8342c06c4a35fa0d6e58d9619fe805fb811fc4e6562fef'

In [5]:
def to_seconds(x):
    seconds=3600*int(x[:2])+60*int(x[3:5])+float(x[6:])
    return seconds

In [6]:
data.call_start_time=data.call_start_time.apply(to_seconds)
data.call_end_time=data.call_end_time.apply(to_seconds)

In [7]:
data["duration"]=3600*24*(data.call_end_day-data.call_start_day)+data.call_end_time-data.call_start_time

In [8]:
del data["call_start_time"]
del data['call_end_time']
del data['call_end_day']
del data['call_start_day']

In [9]:
print(data.head())
data1=data
len(data)

                                              hashed  duration
0  967393e81d99ce8e577ee130b7ce8e4fd45e3e9cecb560...  1591.642
1  02181a0c962f34f019bc9d5b582fb0ec79b1441f96aa4d...  7357.339
2  86022904c5cf72a54978479c94041f4256d6c3c2a1f71c...   615.005
3  aafb40d212fe18ff4eafb82fdcf3b53f2161cb3ce59de4...  1538.256
4  c87c2fad141edf323f3787335b54be22945a02fe052448...  1293.822


6962341

In [26]:
hashed = (i for i in data1['hashed'])
duration = (i for i in data1['duration'])

sum_total = [{},{}]

for i in trange(len(data1)):
    has= next(hashed)
    dur= next(duration)
    happy= (i for i in has.split(','))
    for j in range(len(has.split(','))):
        temp_hashed=next(happy)
        if temp_hashed in sum_total[0].keys():
            sum_total[0][temp_hashed]+=1
        else:
            sum_total[0][temp_hashed]=1
            
        if temp_hashed in sum_total[1].keys():
            sum_total[1][temp_hashed]+= dur
        else:
            sum_total[1][temp_hashed]= dur

    

100%|██████████| 6962341/6962341 [01:08<00:00, 101091.54it/s]


In [27]:
data=pd.DataFrame(sum_total).T

In [28]:
data=data.reset_index()
data.head()

,index,0,1
0,000020b4fa2af1e40c813436e5054bce70b703d0039dcd...,1049.0,1714980.485
1,0000264b01392acfde44f9d8494f112a701dc5d3e5fda6...,777.0,1917417.648
2,00006916672106f94f68d18e4a0532b53a40f4549d3a62...,9.0,4181.431
3,0000988c5778e9d6a861fee8fbfd88ec2fdc9108c1acd6...,37.0,52531.516
4,0000a2ff91f6891797708f346d8e3790fa75c1872d6fdc...,222.0,193136.402


In [32]:
data=data.rename(columns={'index':'id', 0:'count', 1:'total_time'})

In [33]:
data["mean_time"]=data["total_time"]/data["count"]

In [34]:
print(data.head())

                                                  id   count   total_time  \
0  000020b4fa2af1e40c813436e5054bce70b703d0039dcd...  1049.0  1714980.485   
1  0000264b01392acfde44f9d8494f112a701dc5d3e5fda6...   777.0  1917417.648   
2  00006916672106f94f68d18e4a0532b53a40f4549d3a62...     9.0     4181.431   
3  0000988c5778e9d6a861fee8fbfd88ec2fdc9108c1acd6...    37.0    52531.516   
4  0000a2ff91f6891797708f346d8e3790fa75c1872d6fdc...   222.0   193136.402   

     mean_time  
0  1634.871768  
1  2467.718981  
2   464.603444  
3  1419.770703  
4   869.983793  
